In [1]:
import os
import torch
import torch.utils.data
import pickle
import numpy as np
import random

In [2]:
class CustomDataPreprocessorForCNN():
    def __init__(self, input_seq_length=5, pred_seq_length=5, datasets=[i for i in range(37)], test_data_sets = [2], dev_ratio_to_test_set = 0.1, forcePreProcess=False):
        '''
        Initializer function for the CustomDataSetForCNN class
        params:
        input_seq_length : input sequence length to be considered
        output_seq_length : output sequence length to be predicted
        datasets : The indices of the datasets to use
        test_data_sets : The indices of the test sets from datasets
        dev_ratio_to_test_set : ratio of the validation set size to the test set size
        forcePreProcess : Flag to forcefully preprocess the data again from csv files
        '''
        # List of data directories where raw data resides
        self.data_dirs = ['./data/train/processed/biwi/biwi_hotel', './data/train/processed/crowds/arxiepiskopi1',
                          './data/train/processed/crowds/crowds_zara02', './data/train/processed/crowds/crowds_zara03',
                          './data/train/processed/crowds/students001', './data/train/processed/crowds/students003', 
                          './data/train/processed/stanford/bookstore_0',
                          './data/train/processed/stanford/bookstore_1', './data/train/processed/stanford/bookstore_2',
                          './data/train/processed/stanford/bookstore_3', './data/train/processed/stanford/coupa_3',
                          './data/train/processed/stanford/deathCircle_0', './data/train/processed/stanford/deathCircle_1',
                          './data/train/processed/stanford/deathCircle_2', './data/train/processed/stanford/deathCircle_3',
                          './data/train/processed/stanford/deathCircle_4', './data/train/processed/stanford/gates_0',
                          './data/train/processed/stanford/gates_1', './data/train/processed/stanford/gates_3',
                          './data/train/processed/stanford/gates_4', './data/train/processed/stanford/gates_5',
                          './data/train/processed/stanford/gates_6', './data/train/processed/stanford/gates_7',
                          './data/train/processed/stanford/gates_8', './data/train/processed/stanford/hyang_4',
                          './data/train/processed/stanford/hyang_5', './data/train/processed/stanford/hyang_6',
                          './data/train/processed/stanford/hyang_7', './data/train/processed/stanford/hyang_9',
                          './data/train/processed/stanford/nexus_0', './data/train/processed/stanford/nexus_1',
                          './data/train/processed/stanford/nexus_2', './data/train/processed/stanford/nexus_3',
                          './data/train/processed/stanford/nexus_4', './data/train/processed/stanford/nexus_7',
                          './data/train/processed/stanford/nexus_8', './data/train/processed/stanford/nexus_9']
        train_datasets = datasets
        for dataset in test_data_sets:
            train_datasets.remove(dataset)
        self.train_data_dirs = [self.data_dirs[x] for x in train_datasets]
        self.test_data_dirs = [self.data_dirs[x] for x in test_data_sets]
        
        # Number of datasets
        self.numDatasets = len(self.data_dirs)
        
        # Data directory where the pre-processed pickle file resides
        self.data_dir = './data/train/processed'
        
        # Store the arguments
        self.input_seq_length = input_seq_length
        self.pred_seq_length = pred_seq_length
        
        # Validation arguments
        self.dev_fraction = dev_ratio_to_test_set
        
        # Define the path in which the process data would be stored
        self.processed_train_data_file = os.path.join(self.data_dir, "trajectories_cnn_train.cpkl")
        self.processed_dev_data_file = os.path.join(self.data_dir, "trajectories_cnn_dev.cpkl")
        self.processed_test_data_file = os.path.join(self.data_dir, "trajectories_cnn_test.cpkl")
        
        # If the file doesn't exist or forcePreProcess is true
        if not(os.path.exists(self.processed_train_data_file)) or forcePreProcess:
            print("------ Creating pre-processed training data for CNN ------")
            self.preprocess(self.train_data_dirs, self.processed_train_data_file)
        if not(os.path.exists(self.processed_dev_data_file)) or not(os.path.exists(self.processed_test_data_file)) or forcePreProcess:
            print("------ Creating pre-processed dev & test data for CNN ------")
            self.preprocess(self.test_data_dirs, self.processed_test_data_file, self.dev_fraction, self.processed_dev_data_file)
        
    def preprocess(self, data_dirs, data_file, dev_fraction = 0., data_file_2 = None):
        #frameList_data = []
        #pedsInFrameList_data = []
        #pedsPosInFrameList_data = []
        processed_input_output_pairs = []
        
        for directory in data_dirs:
            print('------ Processing dataset ' + str(directory) + ' ------')
            # define path of the csv file of the current dataset
            file_path = os.path.join(directory, 'world_pos_normalized.csv')
            
            # Load the data from the csv file
            data = np.genfromtxt(file_path, delimiter=',')
            
            # Frame IDs of the frames in the current dataset
            frameList = np.unique(data[0, :]).tolist()
            numFrames = len(frameList)
            
            # Add the list of frameIDs to the frameList_data
            #frameList_data.append(frameList)
            
            # For this dataset check which pedestrians exist in each frame.
            pedsInFrameList = []
            pedsPosInFrameList = []
            for ind, frame in enumerate(frameList):
                # For this frame check the pedestrian IDs.
                pedsInFrame = data[:, data[0, :] == frame]
                pedsList = pedsInFrame[1, :].tolist()
                pedsInFrameList.append(pedsList)
                # Position information for each pedestrian.
                pedsPos = []
                for ped in pedsList:
                    # Extract x and y positions
                    current_x = pedsInFrame[2, pedsInFrame[1, :] == ped][0]
                    current_y = pedsInFrame[3, pedsInFrame[1, :] == ped][0]
                    pedsPos.extend([current_x, current_y])
                pedsPosInFrameList.append(pedsPos)
            
            # Go over the frames in this data again to extract data.
            ind = 0
            while ind < len(frameList) - (self.input_seq_length + self.pred_seq_length - 1):
                # List of pedestrians in this frame.
                pedsList = pedsInFrameList[ind]
                # Check if same pedestrians exist in the next (input_seq_length + pred_seq_length - 1) frames.
                peds_contained = True
                for ii in range(self.input_seq_length + self.pred_seq_length):
                    if pedsInFrameList[ind + ii] != pedsList:
                        peds_contained = False
                if peds_contained:
                    print(str(int(self.input_seq_length + self.pred_seq_length)) + ' frames starting from Frame ' + str(int(frameList[ind])) +  ' contain pedestrians ' + str(pedsList))
                    # Initialize numpy arrays for input-output pair
                    data_input = np.zeros((2*len(pedsList), self.input_seq_length))
                    data_output = np.zeros((2*len(pedsList), self.pred_seq_length))
                    for ii in range(self.input_seq_length):
                        data_input[:, ii] = np.array(pedsPosInFrameList[ind + ii])
                    for jj in range(self.pred_seq_length):
                        data_output[:, jj] = np.array(pedsPosInFrameList[ind + (self.input_seq_length - 1) + jj])
                    processed_pair = (torch.from_numpy(data_input), torch.from_numpy(data_output))
                    processed_input_output_pairs.append(processed_pair)
                    
                    # Perform data augmentation. Rotate (x,y)-coordinates from 5 deg to 355 deg with 5 deg space, and flip y. The amount of data is x144 the original data.
                    # # First, flip the original data.
                    data_input_yflipped = np.zeros_like(data_input)
                    data_output_yflipped = np.zeros_like(data_output)
                    for kk in range(len(pedsList)):
                        data_input_yflipped[2*kk, :] = data_input[2*kk, :]
                        data_input_yflipped[2*kk+1, :] = -1*data_input[2*kk+1, :]
                        data_output_yflipped[2*kk, :] = data_output[2*kk, :]
                        data_output_yflipped[2*kk+1, :] = -1*data_output[2*kk+1, :]
                    processed_pair_yflipped = (torch.from_numpy(data_input_yflipped), torch.from_numpy(data_output_yflipped))
                    processed_input_output_pairs.append(processed_pair_yflipped)
                    # # Then rotate by 5 deg sequentially and also flip for each rotated data
                    for deg in range(5, 360, 5):
                        data_input_rotated = np.zeros_like(data_input)
                        data_input_rotated_yflipped = np.zeros_like(data_input)
                        data_output_rotated = np.zeros_like(data_output)
                        data_output_rotated_yflipped = np.zeros_like(data_output)
                        rad = np.radians(deg)
                        c, s = np.cos(rad), np.sin(rad)
                        Rot = np.array(((c,-s), (s, c)))
                        for ii in range(len(pedsList)):
                            for jj in range(self.input_seq_length):
                                coordinates_for_this_ped = data_input[2*ii:2*(ii+1), jj]
                                new_coordinates_for_this_ped = np.dot(Rot, coordinates_for_this_ped)
                                data_input_rotated[2*ii:2*(ii+1), jj] = new_coordinates_for_this_ped
                            data_input_rotated_yflipped[2*ii, :] = data_input_rotated[2*ii, :]    
                            data_input_rotated_yflipped[2*ii+1, :] = -1*data_input_rotated[2*ii+1, :]
                            for jj in range(self.pred_seq_length):
                                coordinates_for_this_ped = data_output[2*ii:2*(ii+1), jj]
                                new_coordinates_for_this_ped = np.dot(Rot, coordinates_for_this_ped)
                                data_output_rotated[2*ii:2*(ii+1), jj] = new_coordinates_for_this_ped
                            data_output_rotated_yflipped[2*ii, :] = data_output_rotated[2*ii, :]
                            data_output_rotated_yflipped[2*ii+1, :] = -1*data_output_rotated[2*ii+1, :]
                        processed_pair_rotated = (torch.from_numpy(data_input_rotated), torch.from_numpy(data_output_rotated))
                        processed_input_output_pairs.append(processed_pair_rotated)   
                        processed_pair_rotated_yflipped = (torch.from_numpy(data_input_rotated_yflipped), torch.from_numpy(data_output_rotated_yflipped))
                        processed_input_output_pairs.append(processed_pair_rotated_yflipped)
                        
                    ind += self.input_seq_length +  self.pred_seq_length - 1
                else:
                    ind += 1
                
            #pedsInFrameList_data.append(pedsInFrameList)
            #pedsPosInFrameList_data.append(pedsPosInFrameList)
            
        # Shuffle data, possibly divide into train and dev sets.
        random.seed(1)
        random.shuffle(processed_input_output_pairs)
        if dev_fraction != 0.:
            assert(data_file_2 != None)
            dev_size = int(len(processed_input_output_pairs)*dev_fraction)
            processed_dev_set = processed_input_output_pairs[:dev_size]
            processed_test_set = processed_input_output_pairs[dev_size:]
            # Save processed data.
            f = open(data_file, 'wb')
            pickle.dump(processed_test_set, f, protocol=2)
            f.close()
            f2 = open(data_file_2, 'wb')
            pickle.dump(processed_dev_set, f2, protocol=2)
            f2.close()
        else:
            # Save processed data.
            f = open(data_file, 'wb')
            pickle.dump(processed_input_output_pairs, f, protocol=2)
            f.close()

In [3]:
processed = CustomDataPreprocessorForCNN(forcePreProcess=True, test_data_sets=[2], dev_ratio_to_test_set=0.3)

------ Creating pre-processed training data for CNN ------
------ Processing dataset ./data/train/processed/biwi/biwi_hotel ------
10 frames starting from Frame 0 contain pedestrians [5.0, 6.0, 8.0]
10 frames starting from Frame 90 contain pedestrians [5.0, 6.0, 8.0]
10 frames starting from Frame 590 contain pedestrians [24.0, 25.0, 28.0]
10 frames starting from Frame 1000 contain pedestrians [38.0]
10 frames starting from Frame 1200 contain pedestrians [46.0, 47.0, 39.0]
10 frames starting from Frame 1400 contain pedestrians [40.0]
10 frames starting from Frame 1490 contain pedestrians [40.0]
10 frames starting from Frame 1600 contain pedestrians [41.0]
10 frames starting from Frame 1690 contain pedestrians [41.0]
10 frames starting from Frame 1800 contain pedestrians [42.0]
10 frames starting from Frame 1890 contain pedestrians [42.0]
10 frames starting from Frame 2770 contain pedestrians [71.0, 72.0]
10 frames starting from Frame 2860 contain pedestrians [71.0, 72.0]
10 frames start

10 frames starting from Frame 72 contain pedestrians [184.0, 188.0, 192.0, 223.0, 186.0, 185.0, 195.0, 190.0]
10 frames starting from Frame 312 contain pedestrians [191.0, 189.0, 193.0, 224.0, 187.0, 196.0]
10 frames starting from Frame 552 contain pedestrians [225.0, 197.0, 201.0, 194.0]
10 frames starting from Frame 792 contain pedestrians [226.0, 198.0, 202.0]
10 frames starting from Frame 996 contain pedestrians [227.0, 203.0, 199.0]
10 frames starting from Frame 5244 contain pedestrians [140.0, 144.0, 148.0, 320.0]
10 frames starting from Frame 5736 contain pedestrians [149.0, 316.0, 146.0, 322.0, 153.0, 376.0]
10 frames starting from Frame 6036 contain pedestrians [317.0, 323.0, 377.0, 312.0, 313.0, 314.0, 340.0]
10 frames starting from Frame 6276 contain pedestrians [318.0, 324.0, 378.0, 315.0, 341.0]
10 frames starting from Frame 6516 contain pedestrians [325.0, 379.0, 342.0]
10 frames starting from Frame 9576 contain pedestrians [12.0, 20.0, 0.0, 2.0, 411.0, 413.0, 415.0, 417.

10 frames starting from Frame 108 contain pedestrians [71.0, 146.0, 148.0, 150.0, 152.0, 154.0, 156.0, 158.0, 160.0, 162.0, 234.0, 248.0, 250.0, 255.0]
10 frames starting from Frame 240 contain pedestrians [72.0, 147.0, 149.0, 151.0, 153.0, 155.0, 157.0, 159.0, 161.0, 163.0, 235.0, 249.0, 251.0, 256.0]
10 frames starting from Frame 348 contain pedestrians [72.0, 147.0, 149.0, 151.0, 153.0, 155.0, 157.0, 159.0, 161.0, 163.0, 235.0, 249.0, 251.0, 256.0]
10 frames starting from Frame 504 contain pedestrians [73.0, 236.0, 252.0, 253.0, 257.0, 22.0, 128.0]
10 frames starting from Frame 2484 contain pedestrians [101.0, 81.0, 30.0, 89.0, 5.0, 121.0]
10 frames starting from Frame 2712 contain pedestrians [102.0, 82.0, 31.0, 90.0, 6.0, 122.0]
10 frames starting from Frame 2952 contain pedestrians [103.0, 83.0, 32.0, 91.0, 7.0]
10 frames starting from Frame 3180 contain pedestrians [104.0, 64.0, 84.0, 33.0, 92.0, 8.0]
------ Processing dataset ./data/train/processed/stanford/gates_3 ------
10 fr

10 frames starting from Frame 288 contain pedestrians [60.0, 72.0, 76.0, 61.0, 64.0, 80.0]
10 frames starting from Frame 528 contain pedestrians [73.0, 77.0, 62.0, 86.0, 65.0, 81.0]
10 frames starting from Frame 768 contain pedestrians [74.0, 78.0, 103.0, 63.0, 87.0, 66.0, 67.0, 82.0]
10 frames starting from Frame 1068 contain pedestrians [104.0, 88.0, 116.0, 83.0, 84.0, 45.0]
10 frames starting from Frame 2076 contain pedestrians [120.0, 49.0, 111.0, 115.0]
10 frames starting from Frame 3084 contain pedestrians [53.0, 56.0, 0.0]
10 frames starting from Frame 3396 contain pedestrians [57.0, 12.0, 7.0]
10 frames starting from Frame 3636 contain pedestrians [13.0, 8.0]
10 frames starting from Frame 3924 contain pedestrians [14.0, 9.0, 17.0, 22.0]
10 frames starting from Frame 4164 contain pedestrians [15.0, 10.0, 18.0, 23.0]
10 frames starting from Frame 4404 contain pedestrians [16.0, 11.0, 19.0, 24.0]
10 frames starting from Frame 4644 contain pedestrians [20.0, 25.0]
10 frames startin

10 frames starting from Frame 0 contain pedestrians [128.0, 130.0, 38.0, 143.0, 147.0, 329.0]
10 frames starting from Frame 108 contain pedestrians [128.0, 130.0, 38.0, 143.0, 147.0, 329.0]
10 frames starting from Frame 264 contain pedestrians [129.0, 131.0, 39.0, 144.0, 148.0, 330.0, 96.0, 105.0]
10 frames starting from Frame 504 contain pedestrians [40.0, 145.0, 149.0, 67.0, 97.0, 106.0]
10 frames starting from Frame 744 contain pedestrians [41.0, 146.0, 150.0, 68.0, 98.0, 107.0]
10 frames starting from Frame 2064 contain pedestrians [10.0, 103.0, 112.0, 314.0, 92.0, 200.0, 206.0]
10 frames starting from Frame 4740 contain pedestrians [22.0, 48.0, 49.0, 121.0, 166.0, 250.0, 94.0]
10 frames starting from Frame 7260 contain pedestrians [79.0, 191.0, 193.0, 33.0]
10 frames starting from Frame 7500 contain pedestrians [80.0, 34.0, 36.0]
10 frames starting from Frame 7740 contain pedestrians [81.0, 35.0, 37.0]
10 frames starting from Frame 9240 contain pedestrians [132.0, 277.0, 138.0, 28

In [18]:
train_file = open(processed.processed_train_data_file, 'rb')
dev_file = open(processed.processed_dev_data_file, 'rb')
test_file = open(processed.processed_test_data_file, 'rb')

In [19]:
processed.processed_train_data_file

'./data/train/processed/trajectories_cnn_train.cpkl'

In [20]:
train = pickle.load(train_file)
dev = pickle.load(dev_file)
test = pickle.load(test_file)

In [21]:
len(train)

56592

In [22]:
len(dev)/len(train)

0.006856092733955329

In [23]:
len(dev)

388

In [24]:
len(test)

908

In [16]:
class CustomDatasetForCNN(torch.utils.data.Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.file = open(self.file_path, 'rb')
        self.data = pickle.load(self.file)
        self.file.close()
    
    def __getitem__(self, index):
        item = self.data[index]
        return item
    
    def __len__(self):
        return len(self.data)     
        

In [17]:
train_set = CustomDatasetForCNN(processed.processed_train_data_file)

In [18]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=1, shuffle=True)

In [23]:
x, y = train_set.__getitem__(99)

In [24]:
x

tensor([[0.1771, 0.1612, 0.1471, 0.1313, 0.1158],
        [0.6733, 0.6807, 0.6873, 0.6947, 0.6982]], dtype=torch.float64)

In [25]:
next(iter(train_loader))

[tensor([[[ 0.1777,  0.1773,  0.1773,  0.1773,  0.1773],
          [ 0.3633,  0.3771,  0.3907,  0.4044,  0.4180],
          [ 0.1191,  0.1191,  0.1191,  0.1191,  0.1191],
          [ 0.0447,  0.0447,  0.0447,  0.0447,  0.0447],
          [ 0.0983,  0.0983,  0.0983,  0.0983,  0.0983],
          [ 0.0544,  0.0544,  0.0544,  0.0544,  0.0544],
          [ 0.0721,  0.0721,  0.0721,  0.0721,  0.0721],
          [ 0.0598,  0.0598,  0.0598,  0.0598,  0.0598],
          [ 0.0661,  0.0661,  0.0661,  0.0661,  0.0661],
          [ 0.0401,  0.0401,  0.0401,  0.0401,  0.0401],
          [-0.3741, -0.3772, -0.3802, -0.3820, -0.3850],
          [ 0.7355,  0.7404,  0.7452,  0.7528,  0.7576],
          [ 0.1136,  0.1219,  0.1327,  0.1418,  0.1506],
          [ 0.2656,  0.2787,  0.2928,  0.3077,  0.3223],
          [-0.1503, -0.1503, -0.1503, -0.1503, -0.1503],
          [ 0.2184,  0.2184,  0.2184,  0.2184,  0.2184],
          [ 0.1071,  0.1047,  0.1019,  0.0973,  0.0945],
          [ 0.1414,  0.1318,  0

In [22]:
x, y = train_set.__getitem__(9)

In [26]:
len(train_loader)

56592